In [2]:
import requests
import pandas as pd
import numpy as np
import json

In [3]:
#Query all data from a year
data=[]
for i in range(1,30):
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2017","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
data
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


['{"get":"players","parameters":{"league":"39","page":"1","season":"2018"},"errors":[],"results":20,"paging":{"current":1,"total":30},"response":[{"player":{"id":160,"name":"M. Vorm","firstname":"Michel","lastname":"Vorm","age":38,"birth":{"date":"1983-10-20","place":"Nieuwegein","country":"Netherlands"},"nationality":"Netherlands","height":"183 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/160.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2018},"games":{"appearences":2,"lineups":2,"minutes":180,"number":null,"position":"Goalkeeper","rating":"7.050000","captain":false},"substitutes":{"in":0,"out":0,"bench":4},"shots":{"total":0,"on":0},"goals":{"total":0,"c

In [ ]:
#SLICE DATA THAT WASNT CAPTURED

In [4]:
#complete year array
for i in range(30,37):
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2017","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)
    
data

['{"get":"players","parameters":{"league":"39","page":"1","season":"2018"},"errors":[],"results":20,"paging":{"current":1,"total":30},"response":[{"player":{"id":160,"name":"M. Vorm","firstname":"Michel","lastname":"Vorm","age":38,"birth":{"date":"1983-10-20","place":"Nieuwegein","country":"Netherlands"},"nationality":"Netherlands","height":"183 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/160.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2018},"games":{"appearences":2,"lineups":2,"minutes":180,"number":null,"position":"Goalkeeper","rating":"7.050000","captain":false},"substitutes":{"in":0,"out":0,"bench":4},"shots":{"total":0,"on":0},"goals":{"total":0,"c

In [26]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

30

In [31]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [27]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = 35
finalPageItems = 3
for i in range(pagesNum+1):
    
    #To adjust for the last page
    if i == pagesNum: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            print(i,j,pkey)
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

0 0 id
0 0 firstname
0 0 lastname
0 0 age
0 0 nationality
0 0 height
0 0 weight
0 0 injured
0 1 id
0 1 firstname
0 1 lastname
0 1 age
0 1 nationality
0 1 height
0 1 weight
0 1 injured
0 2 id
0 2 firstname
0 2 lastname
0 2 age
0 2 nationality
0 2 height
0 2 weight
0 2 injured
0 3 id
0 3 firstname
0 3 lastname
0 3 age
0 3 nationality
0 3 height
0 3 weight
0 3 injured
0 4 id
0 4 firstname
0 4 lastname
0 4 age
0 4 nationality
0 4 height
0 4 weight
0 4 injured
0 5 id
0 5 firstname
0 5 lastname
0 5 age
0 5 nationality
0 5 height
0 5 weight
0 5 injured
0 6 id
0 6 firstname
0 6 lastname
0 6 age
0 6 nationality
0 6 height
0 6 weight
0 6 injured
0 7 id
0 7 firstname
0 7 lastname
0 7 age
0 7 nationality
0 7 height
0 7 weight
0 7 injured
0 8 id
0 8 firstname
0 8 lastname
0 8 age
0 8 nationality
0 8 height
0 8 weight
0 8 injured
0 9 id
0 9 firstname
0 9 lastname
0 9 age
0 9 nationality
0 9 height
0 9 weight
0 9 injured
0 10 id
0 10 firstname
0 10 lastname
0 10 age
0 10 nationality
0 10 height
0 10 

4 9 age
4 9 nationality
4 9 height
4 9 weight
4 9 injured
4 10 id
4 10 firstname
4 10 lastname
4 10 age
4 10 nationality
4 10 height
4 10 weight
4 10 injured
4 11 id
4 11 firstname
4 11 lastname
4 11 age
4 11 nationality
4 11 height
4 11 weight
4 11 injured
4 12 id
4 12 firstname
4 12 lastname
4 12 age
4 12 nationality
4 12 height
4 12 weight
4 12 injured
4 13 id
4 13 firstname
4 13 lastname
4 13 age
4 13 nationality
4 13 height
4 13 weight
4 13 injured
4 14 id
4 14 firstname
4 14 lastname
4 14 age
4 14 nationality
4 14 height
4 14 weight
4 14 injured
4 15 id
4 15 firstname
4 15 lastname
4 15 age
4 15 nationality
4 15 height
4 15 weight
4 15 injured
4 16 id
4 16 firstname
4 16 lastname
4 16 age
4 16 nationality
4 16 height
4 16 weight
4 16 injured
4 17 id
4 17 firstname
4 17 lastname
4 17 age
4 17 nationality
4 17 height
4 17 weight
4 17 injured
4 18 id
4 18 firstname
4 18 lastname
4 18 age
4 18 nationality
4 18 height
4 18 weight
4 18 injured
4 19 id
4 19 firstname
4 19 lastname
4 19 

9 8 weight
9 8 injured
9 9 id
9 9 firstname
9 9 lastname
9 9 age
9 9 nationality
9 9 height
9 9 weight
9 9 injured
9 10 id
9 10 firstname
9 10 lastname
9 10 age
9 10 nationality
9 10 height
9 10 weight
9 10 injured
9 11 id
9 11 firstname
9 11 lastname
9 11 age
9 11 nationality
9 11 height
9 11 weight
9 11 injured
9 12 id
9 12 firstname
9 12 lastname
9 12 age
9 12 nationality
9 12 height
9 12 weight
9 12 injured
9 13 id
9 13 firstname
9 13 lastname
9 13 age
9 13 nationality
9 13 height
9 13 weight
9 13 injured
9 14 id
9 14 firstname
9 14 lastname
9 14 age
9 14 nationality
9 14 height
9 14 weight
9 14 injured
9 15 id
9 15 firstname
9 15 lastname
9 15 age
9 15 nationality
9 15 height
9 15 weight
9 15 injured
9 16 id
9 16 firstname
9 16 lastname
9 16 age
9 16 nationality
9 16 height
9 16 weight
9 16 injured
9 17 id
9 17 firstname
9 17 lastname
9 17 age
9 17 nationality
9 17 height
9 17 weight
9 17 injured
9 18 id
9 18 firstname
9 18 lastname
9 18 age
9 18 nationality
9 18 height
9 18 weigh

13 19 nationality
13 19 height
13 19 weight
13 19 injured
14 0 id
14 0 firstname
14 0 lastname
14 0 age
14 0 nationality
14 0 height
14 0 weight
14 0 injured
14 1 id
14 1 firstname
14 1 lastname
14 1 age
14 1 nationality
14 1 height
14 1 weight
14 1 injured
14 2 id
14 2 firstname
14 2 lastname
14 2 age
14 2 nationality
14 2 height
14 2 weight
14 2 injured
14 3 id
14 3 firstname
14 3 lastname
14 3 age
14 3 nationality
14 3 height
14 3 weight
14 3 injured
14 4 id
14 4 firstname
14 4 lastname
14 4 age
14 4 nationality
14 4 height
14 4 weight
14 4 injured
14 5 id
14 5 firstname
14 5 lastname
14 5 age
14 5 nationality
14 5 height
14 5 weight
14 5 injured
14 6 id
14 6 firstname
14 6 lastname
14 6 age
14 6 nationality
14 6 height
14 6 weight
14 6 injured
14 7 id
14 7 firstname
14 7 lastname
14 7 age
14 7 nationality
14 7 height
14 7 weight
14 7 injured
14 8 id
14 8 firstname
14 8 lastname
14 8 age
14 8 nationality
14 8 height
14 8 weight
14 8 injured
14 9 id
14 9 firstname
14 9 lastname
14 9 

18 7 height
18 7 weight
18 7 injured
18 8 id
18 8 firstname
18 8 lastname
18 8 age
18 8 nationality
18 8 height
18 8 weight
18 8 injured
18 9 id
18 9 firstname
18 9 lastname
18 9 age
18 9 nationality
18 9 height
18 9 weight
18 9 injured
18 10 id
18 10 firstname
18 10 lastname
18 10 age
18 10 nationality
18 10 height
18 10 weight
18 10 injured
18 11 id
18 11 firstname
18 11 lastname
18 11 age
18 11 nationality
18 11 height
18 11 weight
18 11 injured
18 12 id
18 12 firstname
18 12 lastname
18 12 age
18 12 nationality
18 12 height
18 12 weight
18 12 injured
18 13 id
18 13 firstname
18 13 lastname
18 13 age
18 13 nationality
18 13 height
18 13 weight
18 13 injured
18 14 id
18 14 firstname
18 14 lastname
18 14 age
18 14 nationality
18 14 height
18 14 weight
18 14 injured
18 15 id
18 15 firstname
18 15 lastname
18 15 age
18 15 nationality
18 15 height
18 15 weight
18 15 injured
18 16 id
18 16 firstname
18 16 lastname
18 16 age
18 16 nationality
18 16 height
18 16 weight
18 16 injured
18 17 i

22 15 id
22 15 firstname
22 15 lastname
22 15 age
22 15 nationality
22 15 height
22 15 weight
22 15 injured
22 16 id
22 16 firstname
22 16 lastname
22 16 age
22 16 nationality
22 16 height
22 16 weight
22 16 injured
22 17 id
22 17 firstname
22 17 lastname
22 17 age
22 17 nationality
22 17 height
22 17 weight
22 17 injured
22 18 id
22 18 firstname
22 18 lastname
22 18 age
22 18 nationality
22 18 height
22 18 weight
22 18 injured
22 19 id
22 19 firstname
22 19 lastname
22 19 age
22 19 nationality
22 19 height
22 19 weight
22 19 injured
23 0 id
23 0 firstname
23 0 lastname
23 0 age
23 0 nationality
23 0 height
23 0 weight
23 0 injured
23 1 id
23 1 firstname
23 1 lastname
23 1 age
23 1 nationality
23 1 height
23 1 weight
23 1 injured
23 2 id
23 2 firstname
23 2 lastname
23 2 age
23 2 nationality
23 2 height
23 2 weight
23 2 injured
23 3 id
23 3 firstname
23 3 lastname
23 3 age
23 3 nationality
23 3 height
23 3 weight
23 3 injured
23 4 id
23 4 firstname
23 4 lastname
23 4 age
23 4 nationali

27 1 injured
27 2 id
27 2 firstname
27 2 lastname
27 2 age
27 2 nationality
27 2 height
27 2 weight
27 2 injured
27 3 id
27 3 firstname
27 3 lastname
27 3 age
27 3 nationality
27 3 height
27 3 weight
27 3 injured
27 4 id
27 4 firstname
27 4 lastname
27 4 age
27 4 nationality
27 4 height
27 4 weight
27 4 injured
27 5 id
27 5 firstname
27 5 lastname
27 5 age
27 5 nationality
27 5 height
27 5 weight
27 5 injured
27 6 id
27 6 firstname
27 6 lastname
27 6 age
27 6 nationality
27 6 height
27 6 weight
27 6 injured
27 7 id
27 7 firstname
27 7 lastname
27 7 age
27 7 nationality
27 7 height
27 7 weight
27 7 injured
27 8 id
27 8 firstname
27 8 lastname
27 8 age
27 8 nationality
27 8 height
27 8 weight
27 8 injured
27 9 id
27 9 firstname
27 9 lastname
27 9 age
27 9 nationality
27 9 height
27 9 weight
27 9 injured
27 10 id
27 10 firstname
27 10 lastname
27 10 age
27 10 nationality
27 10 height
27 10 weight
27 10 injured
27 11 id
27 11 firstname
27 11 lastname
27 11 age
27 11 nationality
27 11 heigh

In [28]:
finalArr
#len(proto)

587

In [29]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[160, 'Michel', 'Vorm', ..., 0, 0, 0],
       [1444, 'Stephan', 'Lichtsteiner', ..., 0, 0, None],
       [18757, 'Leighton', 'Baines', ..., 0, 0, None],
       ...,
       [2792, 'Aron Einar', 'Malmquist Gunnarsson', ..., 0, 0, None],
       [18827, 'Carlos Alberto', 'Sánchez Moreno', ..., 0, 0, None],
       [73822, 'Marcelo Amado', 'Djaló Taritolay', ..., None, None, None]],
      dtype=object)

In [32]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,160,Michel,Vorm,38,Netherlands,183 cm,84 kg,False,47,Tottenham,...,0,0,0,0,0,None,None,0,0,0
1,1444,Stephan,Lichtsteiner,37,Switzerland,182 cm,76 kg,False,42,Arsenal,...,10,11,3,0,0,None,None,0,0,None
2,18757,Leighton,Baines,37,England,170 cm,74 kg,False,45,Everton,...,1,2,0,0,0,None,2,0,0,None
3,18774,Wes,Morgan,37,Jamaica,186 cm,101 kg,False,46,Leicester,...,6,18,3,2,0,None,None,0,0,None
4,18789,Heurelho,da Silva Gomes,40,Brazil,191 cm,91 kg,False,38,Watford,...,None,None,0,0,0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,298,Rafael Euclides,Soares Camacho,22,Portugal,175 cm,65 kg,False,40,Liverpool,...,0,0,0,0,0,None,None,0,0,None
583,624,Nicolás Hernán Gonzalo,Otamendi,34,Argentina,183 cm,76 kg,False,50,Manchester City,...,3,8,1,0,0,None,None,0,0,None
584,2792,Aron Einar,Malmquist Gunnarsson,33,Iceland,177 cm,95 kg,False,43,Cardiff,...,30,26,5,0,0,None,None,0,0,None
585,18827,Carlos Alberto,Sánchez Moreno,36,Colombia,182 cm,82 kg,False,48,West Ham,...,1,3,0,0,0,None,None,0,0,None


In [33]:
gb = df.groupby(df['lastname']).count()#=='Kane'].goalAssists
df[df['lastname']=='Vorm']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,160,Michel,Vorm,38,Netherlands,183 cm,84 kg,False,47,Tottenham,...,0,0,0,0,0,None,None,0,0,0


In [34]:
df['season'] = '2017/2018'

In [35]:
#Export the df to a csv
df.to_csv('data2017&2018.csv')